### important: whenever you make changes to an imported notebook you need to select "restart the kernel" in this notebook to load the changes then rerun the cells

In [ ]:
# ---------- global parameters

TIMER_ON = True
REFERENCE_MISSION = 2
mission_numbers = [REFERENCE_MISSION, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14] # TODO run Pix4D on mission 7 and on friday mission(s)
DEBUG_IMAGES = True

# ---------- Anomaly Detection

MASK_SETTINGS_ALEX = (75, 175, 1, 2, 7, 1) # (lower limit, upper limit, erode radius, erode iterations, dilate radius, dilate iterations)

# ---------- Heatmap

NUM_BUCKETS = 10
# HEATMAP_SENSITIVITY_MIN = 15 # TODO
# HEATMAP_SENSITIVITY_MAX = 37 # TODO

# ---------- Visualization

BBOX_COLORS = [(0, 0, 255), (255, 0, 0)] # red, blue
NUM_BBOX_ITERATIONS = 2
VOTING_SCHEME = 'AND' # AND, OR, MAJORITY
MAJORITY_SUCCESS = 2 # used for majority, number of detection methods that need to agree

# ----------

In [ ]:
import import_ipynb

import AlexFunctions as AF
import JoshFunctions as JF
import KyleFunctions as KF
# import MasonFunctions as MF # TODO

import LoadData as LD
import ImagePreprocessing as IP
import VisualizeResults as VR

import numpy as np
import time
import sys

# Preprocess New Mission

In [ ]:
new_mission = -1

if new_mission != -1:
    if new_mission in mission_numbers:
        sys.exit('Mission number is already included.')

    if IP.run_image_preprocessor(mission_number=new_mission, save_results=True, display_results=False):
        mission_numbers.append(new_mission)
        print('Mission Numbers:', mission_numbers)
    else:
        sys.exit('Data for mission number ' + str(new_mission) + ' does not exist.')

# Load Data

In [ ]:
if len(mission_numbers) < 2:
    sys.exit('Minimum of 2 missions required.')

In [ ]:
start = time.time()

# load all of the images and regions (bgr, hsv, histogram)
data = LD.run_load_data(mission_numbers, display_images=False)

# original color images
mission_images_bgr = data[0]
grass_images_bgr = data[1]
building_images_bgr = data[2]

# histogram normalized color images
histogram_mission_images_bgr = data[3]
histogram_grass_images_bgr = data[4]
histogram_building_images_bgr = data[5]

# original hsv images
mission_images_hsv = data[6]
grass_images_hsv = data[7]
building_images_hsv = data[8]

# histogram normalized hsv images
histogram_mission_images_hsv = data[9]
histogram_grass_images_hsv = data[10]
histogram_building_images_hsv = data[11]

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

# Anomaly Detection

In [ ]:
start = time.time()

# Alex - anomaly detection
anomaly_masks_1 = AF.anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_hsv, mask_settings=MASK_SETTINGS_ALEX, display_images=False)

# Kyle - anomaly detection
anomaly_masks_2 = KF.anomaly_detection(grass_images_bgr, 45)

# AF.plot_images(image_list=anomaly_masks_1)
# AF.plot_images(image_list=anomaly_masks_2)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

# Grass Region Heatmap

In [ ]:
start = time.time()

# Kyle - heatmap
# TODO comment out print statements in function
heatmap,percents, masklist, color_buckets = KF.score_missions(histogram_grass_images_bgr, NUM_BUCKETS)
# print(percents)
# AF.plot_images(image_list=masklist)
# AF.plot_images(image_list=heatmap)
# for each in percents:
#      print(np.sum(each))

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

# Roof

In [ ]:
start = time.time()

roof_masks, roof_anomaly_percentages = JF.findRoofAnomalies(histogram_building_images_hsv)
# AF.plot_images(roof_masks)
# print(roof_anomaly_percentages)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

# Visualize Results

In [ ]:
start = time.time()

# TODO add heatmap mask and Mason mask
VR.run_visualize_results(mission_numbers, [anomaly_masks_1, anomaly_masks_2], histogram_mission_images_bgr, NUM_BBOX_ITERATIONS, BBOX_COLORS, heatmap, percents, roof_masks, roof_anomaly_percentages, color_buckets, voting_scheme=VOTING_SCHEME, majority_success=MAJORITY_SUCCESS, display_images=False, save_images=False)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

# Generate Summary Output File

In [ ]:
# import GenerateReport as GR
# GR.generate_report()